In [1]:
import json
import random

import torch

from embed_llm.generation.evaluation import (
    create_prompt,
    create_prompt_prefix,
    ensure_reproducibility,
)
from embed_llm.generation.metrics import (
    get_approx_em,
)
from embed_llm.models.augmented_model import EmbedAugPipeline


/home/hippolytepilchen/micromamba/envs/llm_embed/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ensure_reproducibility(29)

In [ ]:


#Must have a params json for pipeline
llm_path = '/lustre/scwpod02/client/kyutai-interns/hippop/models/mistral_7B'




run_name = 'Compr2_L16_Cont_MTAConv_2Norm'
# run_name = None

last_ckpt = '010000' # '008500' #'010000' 


device = 'cuda' if torch.cuda.is_available() else 'cpu'

if torch.cuda.device_count() > 1:
    device = torch.device('cuda:0')
    print(f'Using {device} for loading')

w_embeds = True
max_batch_size = 4
instruct_ckpt = None
# instruct_ckpt = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/TrainEmbed_CA_Cont_Distill_Instruct/checkpoints/checkpoint_010000'
# instruct_ckpt ="/lustre/scwpod02/client/kyutai-interns/hippop/tmp/ToyDecompressingTests_LLM_FT_MaxEmb_true_reversed/checkpoints/checkpoint_020000"


Using cuda:0 for loading


In [9]:
pipeline: EmbedAugPipeline = EmbedAugPipeline.load_inference_model(
    llm_path=llm_path,
    ckpt_path= None if run_name is None else ("/lustre/scwpod02/client/kyutai-interns/hippop/tmp/"
    + run_name
    + "/checkpoints/checkpoint_"
    + last_ckpt),
    device=device,
    llm_name="Mistral7B",
    embed_model_name="NVEmbed",  # Not used if pretrainde ckpt available
    max_batch_size=max_batch_size,
    instruct_ckpt=instruct_ckpt,
)


If n_layers is 1, hidden_dim must be equal to out_dim,                 
 but hidden_dim is not equal to out_dim so hidden_dim is set to out_dim
Loading MLP projector


#### QA tests

In [10]:
eval_data = "/lustre/scwpod02/client/kyutai-interns/hippop/processed_data/eval_ReadComp/squad_test.jsonl"
# eval_data = '/lustre/scwpod02/client/kyutai-interns/hippop/processed_data/eval_QA_NVEmbed/nq_open_data.jsonl' # nq_data.jsonl
context = []
questions = []
answers = []


with open(eval_data, "r") as f:
            for line in f:
                data = json.loads(line)
                questions.append(data["question"].strip())

                if isinstance(data["answer"], str):
                    answers.append([data["answer"].strip()])

                elif isinstance(data["answer"], list):
                    answers.append(data["answer"])
                else:
                    raise ValueError("Invalid answer type")
                # Take the first ranked retrieved passage
                context.append(data["passages"][:5])

c = list(zip(questions, context, answers))
fixed_random = random.Random(0.3)
fixed_random.shuffle(c)
questions, context, answers = zip(*c)
        

In [11]:
# print(pipeline.model.mlp_project)
# print('MLP',pipeline.model.mlp_project)
# print('MLP stats', 'Layer 1', torch.mean(pipeline.model.mlp_project.layers[0].layer1.weight),
#       'Layer 2', torch.mean(pipeline.model.mlp_project.layers[0].layer2.weight))
# print('MLP stats', 'Layer 1', torch.std(pipeline.model.mlp_project.layers[0].layer1.weight),
#       'Layer 2', torch.std(pipeline.model.mlp_project.layers[0].layer2.weight))

# import math 

# pipeline.model.mlp_project.layers[0].layer2.weight = torch.nn.Parameter(-pipeline.model.mlp_project.layers[0].layer1.weight)

# torch.nn.init.constant_(pipeline.model.mlp_project.layers[0].rms_norm.weight, 0.6)
# torch.nn.init.kaiming_uniform_(pipeline.model.mlp_project.layers[0].layer1.weight, a=math.sqrt(5))£
# torch.nn.init.kaiming_uniform_(pipeline.model.mlp_project.layers[0].layer2.weight, a=math.sqrt(5))


In [12]:
w_embeds = True
temp = 0
max_tokens = 64
max_embeds = 1
icl_examples = 3
compressed_doc_in_icl = False
icl_w_document = True
query_w_doc = False
max_bs = 4
rag = False

device_count = torch.cuda.device_count()
other_device = device if device_count <= 1 else torch.device("cuda:1")

# temp = [temp] * max_tokens   
# if max_tokens > i_token_to_flip >= 0:
#     temp[i_token_to_flip] = 1000

if w_embeds:
    pipeline.pipeline_args.w_embeds = True
else:
    pipeline.pipeline_args.w_embeds = False
    
prompt_str, to_embed_str = create_prompt_prefix(
    queries=questions,
    answers=[answer[0] for answer in answers],
    docs=None if not icl_w_document else ["\n".join(cont[:max_embeds]) for cont in context],
    max_examples=icl_examples,
    compressed_doc_in_icl=compressed_doc_in_icl,
)

generated_sequences = []

queries = list(questions[icl_examples:])
docs = list(context[icl_examples:])
truths = list(answers[icl_examples:])
queries.reverse()
docs.reverse()
truths.reverse()
texts_to_embed = []
batch_list_prompts = []
for query, doc in zip(queries[:max_bs], docs[:max_bs]):
    batch_list_prompt, text_to_embed = create_prompt(
        prefix_prompt=prompt_str,
        prefix_embed=to_embed_str,
        doc="\n".join(doc[:max_embeds]),
        w_embeds=w_embeds,
        query=query,
        wdoc=query_w_doc
        )
    batch_list_prompts.append(batch_list_prompt)
    texts_to_embed.append(text_to_embed)

# for i, cont in enumerate(context_prompt if pipeline.pipeline_args.w_prefix_prompt else no_context_prompt):
#     print(f'{i}',cont)


    # print(no_context_prompt[i] if not pipeline.pipeline_args.w_prefix_prompt else context_prompt[i])
    # print(f'Ground truth answer: {a}\n')

generated_sequence = pipeline.generate(
    text_to_embed=texts_to_embed,
    batch_list_prompts=batch_list_prompts,
    temperature=temp,
    max_tokens=max_tokens,
    truncate_line=True,
    device=device,
    device_generation=other_device,
)

print('GENERATION')
for i, (q, a, d, g) in enumerate(zip(queries[:max_bs], truths[:max_bs], docs[:max_bs], generated_sequence)):
    print(f'Given Query: {q}\nGT Answer: {a}\nContext Given {d[:max_embeds]}[...]\nAnswer in context? {get_approx_em(d,a[0])}','\nPrediction:', g, '\n\n')



GENERATION
Given Query: What could someone be investigated for?
GT Answer: ['allegations of professional misconduct', 'professional misconduct', 'professional misconduct']
Context Given ["The functions of the teacher's colleges may include setting out clear standards of practice, providing for the ongoing education of teachers, investigating complaints involving members, conducting hearings into allegations of professional misconduct and taking appropriate disciplinary action and accrediting teacher education programs. In many situations teachers in publicly funded schools must be members in good standing with the college, and private schools may also require their teachers to be college peoples. In other areas these roles may belong to the State Board of Education, the Superintendent of Public Instruction, the State Education Agency or other governmental bodies. In still other areas Teaching Unions may be responsible for some or all of these duties."][...]
Answer in context? 1 
Predic

In [ ]:
# conditioning = ['Kyutai is a non-profit laboratory dedicated to open research in AI, founded in November 2023 by the iliad Group, CMA CGM and Schmidt Sciences. Launched with an initial team of six leading scientists, who have all worked with Big Tech labs in the USA, Kyutai continues to recruit at the highest level, and also offers internships to research Master’s degree students.']*4
# prompts = ['who are the founders of Kyutai?', 'when was Kyutai founded?', 'how many scientists were in the initial team?', 'what does Kyutai offer to research Master’s degree students?']
